# Pre-process Biblioteche Mibact

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import sys

root_dir = os.path.dirname(os.path.dirname(os.getcwd())) ## TODO: find way to put this into some global settings
if root_dir not in sys.path:
    sys.path.append(root_dir)

from references import common_cfg, istat_kpi
from src.models.factories import LibraryFactory

In [ ]:
# Load raw data
with open('../../data/raw/cultura/biblioteche_mibact.json') as json_data:
    d = json.load(json_data)
    
# Prendi le biblioteche
data = d['biblioteche']

In [ ]:
#len(data)#.keys
df = pd.DataFrame.from_records(data)

In [ ]:
data[0]

### Identificazione caratteristiche di interesse

|`accesso.portatori-handicap`|`accesso.riservato`|`cosice-identificativi.isil`|`denominazioni.ufficiale`|`ente`|`indirizzo.coordinate`|`indirizzo.via-piazza`|`tipologia-amministrativa`|`tipologia-funzionale`|
|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|
|True|True|IT-AG0001|Biblioteca della Soprintendenza per i beni cul|Regione siciliana. Soprintendenza per i beni c|[37.3361677, 13.5880717]|viao ciao|comunale|pubblica|

In [ ]:
# Colonne scelte e costruite in modo che il valore che voglio è definito ralla chiave che lo identifica: key.value
colonne = ['accesso.portatori-handicap','accesso.riservato','codici-identificativi.isil','denominazioni.ufficiale','ente','indirizzo.coordinate','indirizzo.via-piazza','tipologia-amministrativa','tipologia-funzionale']

In [ ]:
# Inizializza nuovo df
df = pd.DataFrame()

# Popola df
for c in colonne:
    # Distinguo tra chiave e valore che voglio
    split_colonna = c.split('.')
    
    # Se c'è la distinzione chiave valore
    if len(split_colonna)>1:
        df[c] = np.array([data[i][c.split('.')[0]][c.split('.')[1]] for i in range(len(data))])
    
    else:
        df[c] = np.array([data[i][c] for i in range(len(data))])

In [ ]:
df = df[['codici-identificativi.isil', 'denominazioni.ufficiale', 'ente',
       'indirizzo.coordinate', 'indirizzo.via-piazza',
       'tipologia-amministrativa', 'tipologia-funzionale','accesso.portatori-handicap', 'accesso.riservato']]

In [ ]:
to_drop = []
for i,j in enumerate(df['indirizzo.coordinate']):
    if len(j) != 2:
        to_drop += [i]

In [ ]:
df.drop(to_drop, inplace=True)

In [ ]:
df['Lat'] = df['indirizzo.coordinate'].apply(lambda x: x[0])
df['Long'] = df['indirizzo.coordinate'].apply(lambda x: x[1])
del df['indirizzo.coordinate']

In [ ]:
df['comune'] = [data[i]['indirizzo']['comune']['nome'] for i in range(len(df))]

In [ ]:
# set index with the right name
df.rename({'codici-identificativi.isil': LibraryFactory.id_col}, axis=1, inplace=True)
df.set_index(LibraryFactory.id_col, inplace=True)

In [ ]:
df.head()

In [ ]:
for cittaScelta in common_cfg.city_list:    
    # export to file
    df[df['comune'] == cittaScelta].to_csv(
        '../../data/processed/%s_biblioteche.csv'%cittaScelta, sep=';', decimal=',')